In [12]:
s = '/home/mengkjin/Workspace/learndl/src_factor_definition/level0/highfreq/hf_correlation.py'
from pathlib import Path
[p for p in Path(s).rglob('*.py')]

[]

In [1]:
%run src_runs/autorun/daily_update.py

src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']
Basic module imported!
Use device name: NVIDIA GeForce RTX 4090
--------------------------------------------------------------------------------
****************************** DOWNLOAD CORE DATA ******************************
DataBase object [benchmark_ts],[csi2000],[20241121 to 20241218]...... finished! Cost 0.01 secs
DataBase object [information_ts],[calendar],[20241218] save to /home/mengkjin/Workspace/learndl/data/DataBase/DB_information_ts/calendar.feather successfully
DataBase object [information_ts],[change_name],[20241218] save to /home/mengkjin/Workspace/learndl/data/DataBase/DB_information_ts/change_name.feather successfully
DataBase object [information_ts],[description],[20241218] save to /home/mengkjin/Workspace/learndl/data/DataBase/DB_information_ts/description.feather successfully
DataBase object [information_ts],[industry],[20241218] save to /home/mengkjin/Workspace/learndl/data/DataBase/DB_in

In [1]:
from src.api import DataAPI , ModelAPI
DataAPI.update()
ModelAPI.update()

src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']
Basic module imported!
--------------------------------------------------------------------------------
****************************** DOWNLOAD CORE DATA ******************************
DataBase object [trade_ts],[day_limit],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day_limit\2024\day_limit.20241217.feather successfully
DataBase object [trade_ts],[day_moneyflow],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day_moneyflow\2024\day_moneyflow.20241217.feather successfully
DataBase object [trade_ts],[day],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day\2024\day.20241217.feather successfully
DataBase object [trade_ts],[day_val],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\day_val\2024\day_val.20241217.feather successfully


c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\rqdatac\client.py:257: UserWarning: Your account will be expired after  12 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


DataBase object [trade_ts],[min],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\min\2024\min.20241217.feather successfully
rcquant bar min 20241217 success
process other min bars at 20241217 from source rcquant
DataBase object [trade_ts],in],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\5min\2024\5min.20241217.feather successfully
DataBase object [trade_ts],in],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\10min\2024\10min.20241217.feather successfully
DataBase object [trade_ts],in],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\15min\2024\15min.20241217.feather successfully
DataBase object [trade_ts],in],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\30min\2024\30min.20241217.feather successfully
DataBase object [trade_ts],in],[20241217] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\60min\2024\60min.20241217.feather successfully
----------------------

Extract gru_day_V0/1/20241129/best/20241217: 100%|██████████| 314/314 [02:43<00:00,  1.92it/s]


  -->  Finish updating hidden feature extraction for HiddenExtractingModel(hidden_name=gru_day_V0,name=gru_day,submodels=['best'],nums=None) , len=2
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE FACTORS ********************************
model_name is None, update all registered models (len=3)
  -->  Finish updating model prediction for RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0) , len=1
  -->  Finish deploying model prediction for RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0) , len=1
  -->  Finish updating model prediction for RegisteredModel(pred_name=gruRTN_day_V0,name=gruRTN_day,submodel=swalast,num=0) , len=1
  -->  Finish deploying model prediction for RegisteredModel(pred_name=gruRTN_day_V0,name=gruRTN_day,submodel=swalast,num=0) , len=1
  -->  Finish updating model pred

In [1]:
from src.api import DataAPI , ModelAPI
ModelAPI.update()

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!
--------------------------------------------------------------------------------
***************************** PREPARE PREDICT DATA *****************************
predict is True , Data Processing start!
6 datas : ['y', 'day', '30m', 'style', 'indus', 'week'] , from -366 to None
y is up to 20241212003258 already!
day is up to 20241212003259 already!
30m is up to 20241212003302 already!
style is up to 20241212003304 already!
indus is up to 20241212003308 already!
week is up to 20241212003315 already!
Data Processing Finished! Cost 0.00 Seconds
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE HIDDEN ********************************
model_name is None, update all hidden models (len=1)
Beware! Should be at server or short_test, but short_test is

100%|██████████| 316/316 [00:00<00:00, 2575.99it/s]


  -->  Finish updating hidden feature extraction for HiddenExtractingModel(hidden_name=gru_day_V0,name=gru_day,submodels=['best'],nums=None) , len=2
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE FACTORS ********************************
model_name is None, update all registered models (len=3)
  -->  No new updating model prediction for RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0)
  -->  Finish deploying model prediction for RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0) , len=5
  -->  No new updating model prediction for RegisteredModel(pred_name=gruRTN_day_V0,name=gruRTN_day,submodel=swalast,num=0)
  -->  Finish deploying model prediction for RegisteredModel(pred_name=gruRTN_day_V0,name=gruRTN_day,submodel=swalast,num=0) , len=5
  -->  No new updating model prediction for Regis

In [ ]:
from src.basic import RegisteredModel
model = RegisteredModel.SelectModels('gru_day_V0')[0]
model.fmp_target_dates

In [ ]:
from src.api import ModelAPI
model = ModelAPI.Testor('ts_mixer')
model.try_forward()
model.try_metrics()

In [ ]:
from src.basic import RegisteredModel , CONF
from src.model.model_module.application import ModelPortfolioBuilder

model_name = 'gru_day_V0'

model = RegisteredModel.SelectModels(model_name)[0]
md = ModelPortfolioBuilder(model)
md